In [55]:
pip install requests pydantic transformers sentence-transformers scikit-learn google-generativeai==0.5.1 python-dotenv


python(34861) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [56]:
!python --version

python(34862) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Python 3.12.8


In [57]:
! conda install -c conda-forge ipywidgets

python(34863) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/div/miniconda3/lib/python3.12/site-packages/conda/base/context.py:201: FutureWarning: Adding 'defaults' to channel list implicitly is deprecated and will be removed in 25.3. 

To remove this warning, please choose a default channel explicitly with conda's regular configuration system, e.g. by adding 'defaults' to the list of channels:

  conda config --add channels defaults

For more information see https://docs.conda.io/projects/conda/en/stable/user-guide/configuration/use-condarc.html

  deprecated.topic(
/Users/div/miniconda3/lib/python3.12/site-packages/conda/base/context.py:201: FutureWarning: Adding 'defaults' to channel list implicitly is deprecated and will be removed in 25.3. 

To remove this warning, please choose a default channel explicitly with conda's regular configuration system, e.g. by adding 'defaults' to the list of channels:

  conda config --add channels defaults

For more information see https://docs.conda.io/projects/conda/en/stable/user-guide/configuratio

In [58]:
import google.generativeai as genai
import json, re
import os
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

In [59]:
def process_with_gemini(text: str):
    """
    Process input text using the Gemini API and return a structured JSON output.
    """
    prompt = f"""
    Process the following text and return the output in the following JSON structure:
    {{
      "title": "Concise title summarizing the content",
      "key_points": ["Point 1", "Point 2", "..."],
      "sentiment": "Positive/Neutral/Negative",
      "entities": ["Entity1", "Entity2", "..."]
    }}
    Text: {text}
    """
    
    try:
        response = model.generate_content(prompt)
        result = response.text

        title_match = re.search(r'"title":\s*"([^"]+)"', result)
        title = title_match.group(1) if title_match else "N/A"

        key_points_match = re.search(r'"key_points":\s*\[(.*?)\]', result)
        key_points = [point.strip().strip('"') for point in key_points_match.group(1).split(",")] if key_points_match else []

        sentiment_match = re.search(r'"sentiment":\s*"([^"]+)"', result)
        sentiment = sentiment_match.group(1) if sentiment_match else "N/A"

        entities_match = re.search(r'"entities":\s*\[(.*?)\]', result)
        entities = [entity.strip().strip('"') for entity in entities_match.group(1).split(",")] if entities_match else []

        json_output = {
            "title": title,
            "key_points": key_points,
            "sentiment": sentiment,
            "entities": entities
        }
        return json.dumps(json_output, indent=4)  

    except Exception as e:
        print(f"Error processing with Gemini API: {e}")
        return json.dumps({"error": "Failed to process text", "details": str(e)})

In [60]:
from pydantic import BaseModel, ValidationError
from typing import List

class ProcessedData(BaseModel):
    title: str
    key_points: List[str]
    sentiment: str
    entities: List[str]

def validate_response(data: dict):
    try:
        processed = ProcessedData(**data)
        return processed
    except ValidationError as e:
        print("Validation Error:", e.json())
        return None

In [61]:
from transformers import pipeline

def process_with_local_model(text: str):
    model_pipeline = pipeline("text2text-generation", model="t5-large")
    prompt = f"""
    Process the following text and return the output in the following JSON structure:
    {{
      "title": "Concise title summarizing the content",
      "key_points": ["Point 1", "Point 2", "..."],
      "sentiment": "Positive/Neutral/Negative",
      "entities": ["Entity1", "Entity2", "..."]
    }}
    Text: {text}
    """
    # print(prompt)
    response = model_pipeline(
        prompt,
        do_sample=True,
        # max_length=1024,
        # temperature=0.7
    )
    # print(response)
    return response[0]['generated_text']

In [62]:
import time
import random

def api_request_with_retry(request_func, *args, retries=3, backoff_factor=0.3):
    for attempt in range(retries):
        try:
            return request_func(*args)
        except Exception as e:
            if attempt < retries - 1:
                time.sleep(backoff_factor * (2 ** attempt) + random.uniform(0, 0.1))
            else:
                print("API request failed:", e)
                return None

In [63]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

def compare_outputs(api_output, local_output):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    api_embedding = model.encode(" ".join(api_output.get("key_points", [])))
    local_embedding = model.encode(" ".join(local_output.get("key_points", [])))

    similarity = cosine_similarity([api_embedding], [local_embedding])[0][0]
    print("Semantic Similarity:", similarity)

    print("API Sentiment:", api_output.get("sentiment"))
    print("Local Sentiment:", local_output.get("sentiment"))

In [64]:
import os
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "t5-large"
model_path="./t5-large"

if not os.path.exists(model_path):
    # model_pipeline = pipeline("text2text-generation", model="tuner007/pegasus_paraphrase_large")
    # model_pipeline.save_pretrained(model_path)
    localmodel = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    localmodel.save_pretrained(model_path)
    tokenizer.save_pretrained(model_path)


if __name__ == "__main__":
    raw_text = "Apple announced the launch of its new iPhone 15 today. Fans are excited about its enhanced camera and longer battery life. Critics, however, feel that the price point is too high."

    gemini_output = process_with_gemini(raw_text)
    print("Gemini Output:")
    print(gemini_output)
    if isinstance(gemini_output, str):
        gemini_output = json.loads(gemini_output) 

    validated_gemini_output = validate_response(gemini_output)

    local_output = process_with_local_model(raw_text)
    print("Local Output:")
    print(local_output)

    try:
        validated_local_output = validate_response(local_output)
        if validated_gemini_output and validated_local_output:
            compare_outputs(validated_gemini_output.dict(), validated_local_output.dict())
    except Exception as e:
        print("Generated text from local model did not pass the validation", e)


Gemini Output:
{
    "title": "iPhone 15 Launch: Positive Fan Reaction, Price Concerns",
    "key_points": [
        "Apple launched the iPhone 15.",
        "Fans are excited about the improved camera and battery life.",
        "Critics cite high price as a concern."
    ],
    "sentiment": "Neutral",
    "entities": [
        "Apple",
        "iPhone 15"
    ]
}


Device set to use mps:0


Local Output:
iPhone 15 This week, Apple officially introduced a new iPhone. Fans are excited about its improved
Generated text from local model did not pass the validation __main__.ProcessedData() argument after ** must be a mapping, not str
